In [1]:
import os
import librosa
import numpy as np
from PIL import Image
import warnings
import librosa.display
from librosa import feature
import matplotlib.pyplot as plt

warnings.resetwarnings()

In [2]:
SAMPLE_RATE = 16000
FRAME_SIZE = 2048
HOP_SIZE = FRAME_SIZE // 4
SEGMENT_LENGTH = SAMPLE_RATE * 2
SEGMENT_HOP = SEGMENT_LENGTH // 2

In [4]:
input_path = "/Users/langtowl/PycharmProjects/gunshot-detection/data/processed-audio-ngs"
output_path = "/Users/langtowl/PycharmProjects/gunshot-detection/data/spectrograms-ngs"

print(os.path.exists(input_path))
print(os.path.exists(output_path))

True
True


In [5]:
# Figure out how many images to skip

img = 0
for i in range(0, 9847):
    if i % 4 == 0:
        img += 1

print(img)

2462


In [7]:
def mel_spectrogram_generator(path, save_path = "", sr = 16000, duration = 2.0, n_fft = 2560, hop_length = 128, n_mels = 512, fmin = 4000, fmax = 8000, power = 2.0, figsize = (5,5), target_shape = (256, 256), show = False, save = False):
    # Load audio data from path
    data, sr = librosa.load(path, sr = sr, duration = duration)

    # Compute spectrogram
    spectrogram = librosa.feature.melspectrogram(
        y = data,
        sr = sr,
        n_fft = n_fft,
        hop_length = hop_length,
        n_mels = n_mels,
        fmin = fmin,
        fmax = fmax,
        power = power
    )

    # Convert to decibel
    spectrogram_decibel = librosa.power_to_db(spectrogram)

    # Open and configure plot
    fig, ax = plt.subplots(figsize=figsize, dpi=100)
    ax.set_position([0, 0, 1, 1])
    ax.set_frame_on(False)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    fig.patch.set_alpha(0)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.ioff()

    # Plot spectrogram
    librosa.display.specshow(
        spectrogram_decibel,
        sr = sr,
        hop_length = hop_length,
        x_axis = "time",
        y_axis = "mel",
        fmin = fmin,
        fmax = fmax,
        vmin = -20,
        vmax = 10,
        cmap = 'magma'
    )

    # Copy spectrogram graph to image, then delete graph
    fig.canvas.draw()
    image = Image.frombytes('RGB', fig.canvas.get_width_height(), fig.canvas.tostring_rgb())
    plt.close(fig)

    # Resize image to CNN input layer
    image = image.resize(target_shape)

    # When True: Image of spectrogram is saved to cwd
    if save:
        image.save(save_path)

    # When False: Return numpy array for CNN input
    if not show:

        # Convert to array
        image_array = np.array(image)

        # Normalize [0, 1]
        image_array = image_array.astype(np.float32) / 255.0

        return image_array

    # When True: Print spectrogram to console
    else:
        image.show()

In [8]:
# Oh good lord...
print(len(os.listdir(output_path)))

0


In [9]:
print(f"{output_path}/gunshot_{0}.png")

/Users/langtowl/PycharmProjects/gunshot-detection/data/spectrograms-ngs/gunshot_0.png


In [10]:
count = 0

# Loop through files in input_path directory
for file in sorted(os.listdir(input_path)):
    # Increment count for each file
    count += 1

    # Only process every 4th file
    if count % 4 != 0:
        continue

    # Open file path
    file_path = os.path.join(input_path, file)
    file_name = f"{output_path}/{file[:-4]}.png"

    # Generate spectrogram
    mel_spectrogram_generator(file_path, save_path = file_name, save = True)

/Users/langtowl/miniconda3/lib/python3.12/site-packages/librosa/core/intervals.py:15: DeprecationWarning: path is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  with resources.path("librosa.core", "intervals.msgpack") as imsgpack:
/Users/langtowl/miniconda3/lib/python3.12/site-packages/audioread/rawread.py:16: DeprecationWarning: 'aifc' is deprecated and slated for removal in Python 3.13
  import aifc
/Users/langtowl/miniconda3/lib/python3.12/site-packages/audioread/rawread.py:17: DeprecationWarning: 'audioop' is deprecated and slated for removal in Python 3.13
  import audioop
/Users/langtowl/miniconda3/lib/python3.12/site-packages/audioread/rawread.py:19: DeprecationWarning: 'sunau' is deprecated and slated for removal in Python 3.13
  import sunau
/var/folders/bd/gcjm_l21715dj3pty72qkkr80000gn/T/ipykernel_3476/3614600653.py:47: MatplotlibDeprecationWarning: The tostring_rgb funct